# GEEMAP ONLINE AND OFFLINE DATA ACCESS 
## (PART 1 - SINGLE BAND SPATIAL DATA)

Course: CEE5003 Remote Sensing of Land Surfaces  

Created by: Alfonso Torres  

Date: May 2024

***

### Goal 

Demonstrate data access and retrieval when using Google Earth Engine 
***

### Data Sources

* Cloud:  
 a. GEE [ERA5-Land Daily Aggregated - ECMWF Climate Reanalysis](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_DAILY_AGGR)    

*  ERAS5 products of interest

*    snow water equivalent "snow_depth_water_equivalent"	  
*    air temperature at 2 meters "temperature_2m"  
*    evapotranspiration "evaporation_from_vegetation_transpiration_sum"

b. GEE [TIGER: US Census States 2018](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_States)  




* Local:
    None  

***

### Implementation

1. Importing modules

In [30]:
import ee
import geemap
import eemont
import pprint

In [31]:
##### uncomment this cell and run it if the cell below fails
# import geemap
# m = geemap.Map()
# m

2. Initialize Earth Engine

In [32]:
ee.Initialize()

### Accessing Different Raster Products

#### Climate and Weather

3. Load Utah Boundary

In [33]:
boundary_ut =ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME', 'Utah')).first()

In [34]:
#boundary_ut

4. Import ERAS5 Data

In [35]:
ERAS5_daily = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")

5. Clip ERAS5 products of interest for the most recent April 1st

In [36]:
#filtering by date: April 1st.
date = '2021-04-01'

In [37]:
swe_ut = ERAS5_daily.closest(date).select('snow_depth_water_equivalent') .first()

swe_ut = swe_ut.clip(boundary_ut)

In [38]:
temp_ut = ERAS5_daily.closest(date).select('temperature_2m') .first()

temp_ut = temp_ut.clip(boundary_ut)

In [39]:
et0_ut = ERAS5_daily.closest(date).select('potential_evaporation_sum') .first()

et0_ut = et0_ut.clip(boundary_ut)

6. To plot single rasters, we need their min and max values. Creating a function to do that.

In [40]:
# creating a function to estimate mins and maxs of an image

def min_max_values(single_band_image):
# getting the min and max value of a single band image
    meanDictionary = single_band_image.reduceRegion(**{
    'reducer': ee.Reducer.minMax(),
    'geometry': single_band_image.geometry(),
    'bestEffort':True
    })

    max_min_values = meanDictionary.getInfo()
    keys_list = list(max_min_values.keys())

    max_key = keys_list[0]
    min_key = keys_list[1]
    
    maxValue = meanDictionary.get(max_key).getInfo()
    minValue = meanDictionary.get(min_key).getInfo()

    #rounding the numbers
    maxValue = round(maxValue)
    minValue = round(minValue)

    return (maxValue, minValue)

7. Creating a Map

In [41]:
Map1 = geemap.Map()

Map1.centerObject(boundary_ut, 6)	


8. Preparing and displaying ERAS5 snow water equivalent

In [42]:
# getting the min and max value
max_min_swe=min_max_values(swe_ut)

# setting the colorbar from Colorbrewer
color_range = ['#f7fcf0','#e0f3db','#ccebc5','#a8ddb5','#7bccc4','#4eb3d3','#2b8cbe','#0868ac','#084081']

# setting up the value and color range 
singleBandVis = {
    "min": max_min_swe[1],
    "max": max_min_swe[0],
    "palette": color_range
}

# adding a colorbar in the map
palette = singleBandVis["palette"]
vmin = singleBandVis["min"]
vmax = singleBandVis["max"]

Map1.addLayer(swe_ut, singleBandVis, 'UT SWE (meters)')
Map1.add_colorbar_branca(colors=palette, vmin=vmin, vmax=vmax, layer_name="UT SWE (meters)")

In [43]:

Map1

Map(center=[39.30553364092354, -111.67021681825779], controls=(WidgetControl(options=['position', 'transparent…

9. Prepapring and displaying ERAS5 air temperature

In [44]:
# getting the min and max value
max_min_temp=min_max_values(temp_ut)

# setting the colorbar from Colorbrewer
color_range = ['#ffffcc','#ffeda0','#fed976','#feb24c','#fd8d3c','#fc4e2a','#e31a1c','#bd0026','#800026']

# setting up the value and color range 
singleBandVis = {
    "min": max_min_temp[1],
    "max": max_min_temp[0],
    "palette": color_range
}

# adding a colorbar in the map
palette = singleBandVis["palette"]
vmin = singleBandVis["min"]
vmax = singleBandVis["max"]

Map1.addLayer(temp_ut, singleBandVis, 'UT air temp at 2m (Kelvins)')
Map1.add_colorbar_branca(colors=palette, vmin=vmin, vmax=vmax, layer_name="UT air temp at 2m (Kelvins)")

10. Preparing and dispaying ERAS5 potential evapotranspiration

In [45]:
# getting the min and max value
max_min_et=min_max_values(et0_ut)

# setting the colorbar from Colorbrewer
color_range = ['#ffffd9','#edf8b1','#c7e9b4','#7fcdbb','#41b6c4','#1d91c0','#225ea8','#253494','#081d58']

# setting up the value and color range 
singleBandVis = {
    "min": max_min_et[1],
    "max": max_min_et[0],
    "palette": color_range
}

# adding a colorbar in the map
palette = singleBandVis["palette"]
vmin = singleBandVis["min"]
vmax = singleBandVis["max"]

Map1.addLayer(et0_ut, singleBandVis, 'UT evapotranspiration (mm/day)')
Map1.add_colorbar_branca(colors=palette, vmin=vmin, vmax=vmax, layer_name="UT evapotranspiration (mm/day)")